https://mp.weixin.qq.com/s/hcGintcTQtiuAEinghvSbg

# `ELMo`原理

`Embeddings from Language Models`：
- **根据上下文动态调整当前词的词向量表示**，如下例中`apple`两种不同的涵义，对应不同的词向量
```
I like to eat apple
I like apple products
```

`ELMo`采用了典型的两阶段过程：
- 利用大量语料，预训练一个语言模型；该语言模型相当于一个**动态词向量生成器**，用于给具体任务生成词向量；
- 在进行下游任务时，从第一阶段预训练模型中提取对应单词的网络各层的`Word Embedding`作为新特征补充到下游任务中。

## 预训练阶段




<img src="../images/ELMo.jpg" width="80%">
- 模型采用了双向双层`LSTM`，输入为单词或字符的`embedding`
        
        
- 模型的最底层输入$(E_1,E_2,...,E_N)$，代表一句话中N个词的初始词向量，这个词向量是利用字符卷积`(char-cnn)`得到  
- 模型的输出$(T_1,T_2,...,T_N)$，$T_i$代表每个单词出现的概率，即预测位置 i 处的单词为哪一个

   
- 预训练的目标是根据单词$w_i$的上下文去正确预测单词$w_i$；
    - 上图中左边的`LSTM`进行正向的预测，从左到右依次预测句子的单词；给定上文单词$\{t_1,...,t_{k-1}\}$，对单词$t_k$的概率建模，计算序列出现的概率
$$p(t_1,t_2,...,t_N)=\prod_{k=1}^{N}p(t_k|t_1,t_2,...,t_{k-1})$$
    - 右边的逆向双层`LSTM`代表反方向编码器，输入的是从右到左的句子下文。
$$p(t_1,t_2,...,t_N)=\prod_{k=1}^{N}p(t_k|t_{k+1},t_{k+2},...,t_N)$$
    - 模型的目标就是同时最大化前后向语言模型的对数似然
$$\sum_{k=1}^{N}\Big(\big(logp(t_k|t_1,t_2,...,t_{k-1})\big)+\big(logp(t_k|t_{k+1},t_{k+2},...,t_N)\big)\Big)$$

- 因此，对于每个输入的`token`$t_k$，`ELMo`利用`L`层的双向`LSTM`将其表示成`2L+1`个向量：原始的输入向量，及每一层的前向`LSTM`输出，和逆向`LSTM`输出，通常会将前向和逆向输出拼接，作为该层的输出向量
- 原始输入向量表征单词特征，第一层`LSTM`表征句法特征，第二层`LSTM`表征语义特征；越高层，越能捕获词意信息，越能区分一词多义，表示对词义消歧做的越好。

`ELMo`的变体，前向网络和逆向网络不是分开，而是合并后再两层堆叠起来，且使用残差层，将原始输入向量和第一层的输出合并作为第二层的输入
 
<img src="../images/elmo_combination.png" width="80%" alt="elmo">

## 使用语言模型

将每一层的双向`LSTM`产生的向量乘以权重参数，作为下游任务的一种特征输入，将该特征与下游任务的词向量进行拼接，构成最终的词向量进行任务的训练。下游任务训练时，冻结`ELMo`的参数
$$\text{EMLo}_k=\gamma\sum_{j=0}^{L}s_jh_{k,j}$$
$$\text{INPUT}_k=\big[E_k,\text{EMLo}_k\big]$$
其中$s_j$表示每一层输出向量的权重，$h_{k,j}$表示第$k$个单词在第$j$层的输出;

双层的`LSTM`获得的词向量：
    $$\text{ELMo}_k^{task} = \gamma_k \cdot (s_0^{task}\cdot x_{k} + s_1^{task}\cdot h_{1,k} + s_2^{task} \cdot h_{2,k})$$
    
    

# 使用`ELMo`语言模型

In [7]:
import tensorflow_hub as hub
import tensorflow as tf
# # tf.compat.v1.disable_eager_execution()

# elmo = hub.load("../H/tfhub/elmo/")
# texts = ["the cat is on the mat", "dogs are in the fog"]
# embeddings = elmo(texts, signature='default', as_dict=True)["default"]

# # from tensorflow.python.keras import backend as K

# # sess = K.get_session()
# # array = sess.run(embeddings)

elmo = hub.Module("../../H/tfhub/elmo/", trainable=True)
embeddings = elmo(
    ["the cat is on the mat", "dogs are in the fog"],
    signature="default",
    as_dict=True)["elmo"]

RuntimeError: Exporting/importing meta graphs is not supported when eager execution is enabled. No graph exists when eager execution is enabled.